### This part is hovernet preprocess: After MATLAB part of analysis finished on the MATLAB code, this is for modifying some of the wsi(ndpi)src, roisrc, and ndpisrc so that we're only using the 214 image files that we want to use. The MATLAB all_output_concat was manually made by just copy and pasting contents of the each of the excel output files from the MATLAB workflow. After running hovernet workflow, manually add hovernet result to all_output_concat to match 1090 features.
### These src's are used in the hovernet_json2df.py, just making sure all the src's are correct so that hovernet_json2df.py runs properly!

In [16]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import shutil
import pickle
from tqdm import tqdm

In [ ]:
xlsrc = r'\\shelter\Kyu\skin_aging\clue_cohort\CLUE_image_list_230207_v2.xlsx'
ndpisrc = r'\\shelter\Kyu\skin_aging\clue_cohort\wsi'
xl = pd.read_excel(xlsrc)
xl_tmp = xl[xl["student score"] > 1]
wsi_list = xl_tmp.filename
# create ndpi path:
wsi_ndpi_list = [os.path.join(ndpisrc, x) for x in wsi_list]
dst_src = r'\\shelter\Kyu\skin_aging\clue_cohort\wsi\desired_wsi'
for filename in wsi_ndpi_list:
    shutil.copy(filename, dst_src)
# create dl path:
dlsrc = r'\\shelter\Kyu\skin_aging\clue_cohort\DLmask1um'
wsi_list = [x.replace(".ndpi", ".tif") for x in wsi_list]
wsi_dl_list = [os.path.join(dlsrc, x) for x in wsi_list]
dst_src1 = r'\\shelter\Kyu\skin_aging\clue_cohort\DLmask1um\desired_DLmask'
for filename in wsi_dl_list:
    shutil.copy(filename, dst_src1)
# create roi path:
roisrc = r'\\shelter\Kyu\skin_aging\clue_cohort\annotations\roi\labeledmask_v2_021723'
wsi_list = [x.replace(".tif", ".png") for x in wsi_list]
wsi_roi_list = [os.path.join(roisrc, x) for x in wsi_list]
dst_src2 = r'\\shelter\Kyu\skin_aging\clue_cohort\annotations\roi\labeledmask_v2_021723\desired_roi'
for filename in wsi_roi_list:
    shutil.copy(filename, dst_src2)

### This part is hovernet postprocess: This code belows processes the generated .pkl file from hovernet to the .xlsx file in a form that we desire (the average values over each patient's roi):

In [1]:
import os
import pickle
from tqdm import tqdm
import pandas as pd
import numpy as np

In [2]:
pkl_src=r'\\shelter\Kyu\skin_aging\clue_cohort\wsi\hovernet_out\df'
output_src=r'\\shelter\Kyu\skin_aging\clue_cohort\wsi\hovernet_out\df\output_excel'
pkl_list = [x for x in os.listdir(pkl_src) if x.endswith(".pkl")]
pkl_full_list = [os.path.join(pkl_src,x) for x in pkl_list]

In [107]:
# # Code to make the test.xlsx loaded in below: (It's just empty df with column names)
# with open(pkl_full_list[1], 'rb') as f:
#     print("converting pkl to df:")
#     df = pickle.load(f)
#     bbox_tmp = df["bbox"].tolist()
#     for idx1 in range(len(bbox_tmp)):
#         bbox_tmp[idx1] = [[int(x/2),int(y/2)] for (x,y) in bbox_tmp[idx1]]
#     df["bbox_20x"] = bbox_tmp
#     df['centroid_20x'] = df['centroid'].apply(lambda row: [_/2 for _ in row])
#
#     contour_tmp = df["contour"].tolist()
#     for idx2 in range(len(contour_tmp)):
#         contour_tmp[idx2] = [[int(x/2),int(y/2)] for (x,y) in contour_tmp[idx2]]
#     df["contour_20x"] = contour_tmp
# # df dict, key is the type and the value is a DataFrame with the relevant rows
# dfs_by_type = {}
# for t in range(2, 12):
#     dfs_by_type[t] = df[df['type'] == t].reset_index(drop=True)
#
# expanded_dfs = []
# for t, df_t in dfs_by_type.items():
#     new_cols = {f"{col}_type{t}": df_t[col] for col in df_t.columns if col != 'type'}
#     expanded_dfs.append(pd.DataFrame(new_cols))
#
# expanded_df = pd.concat(expanded_dfs, axis=1)
#
# c2t_distance_cols = expanded_df.filter(regex='c2t_distance').columns
# col_name_list = list()
# for col in c2t_distance_cols:
#     for i in range(2, 13):
#         col_name = str(col) + "_" + str(i)
#         col_name_list.append(col_name)
# all_df = pd.DataFrame(columns=col_name_list)
# all_df.to_excel(r"C:\Users\Kevin\Desktop\test.xlsx")

In [ ]:
for idx, pkl in tqdm(enumerate(range(len(pkl_list))),desc="Pickle File Processed",colour='red',total = len(pkl_list)):
    with open(pkl_full_list[idx], 'rb') as f:
        print("converting pkl to df:")
        df = pickle.load(f)
        bbox_tmp = df["bbox"].tolist()
        for idx1 in range(len(bbox_tmp)):
            bbox_tmp[idx1] = [[int(x/2),int(y/2)] for (x,y) in bbox_tmp[idx1]]
        df["bbox_20x"] = bbox_tmp
        df['centroid_20x'] = df['centroid'].apply(lambda row: [_/2 for _ in row])

        contour_tmp = df["contour"].tolist()
        for idx2 in range(len(contour_tmp)):
            contour_tmp[idx2] = [[int(x/2),int(y/2)] for (x,y) in contour_tmp[idx2]]
        df["contour_20x"] = contour_tmp
    # df dict, key is the type and the value is a DataFrame with the relevant rows
    dfs_by_type = {}
    for t in range(2, 12):
        dfs_by_type[t] = df[df['type'] == t].reset_index(drop=True)

    expanded_dfs = []
    for tt, df_t in dfs_by_type.items():
        new_cols = {f"{col}_type{tt}": df_t[col] for col in df_t.columns if col != 'type'}
        expanded_dfs.append(pd.DataFrame(new_cols))

    expanded_df = pd.concat(expanded_dfs, axis=1)
    imID,_ = os.path.splitext(pkl_list[idx])
    output_pth = os.path.join(output_src,imID + ".xlsx")

    # save original df
    expanded_df.to_excel(output_pth, index=False)

    # extract the distances:
    print("extracting distance")
    all_df = pd.read_excel(r"C:\Users\Kevin\Desktop\test.xlsx")
    del all_df[all_df.columns[0]]
    all_df = all_df.reindex(range(expanded_df.shape[0]))
    print("empty row number", expanded_df.shape[0])
    c2t_distance_cols = expanded_df.filter(regex='c2t_distance').columns
    for _, col1 in enumerate(c2t_distance_cols):
        print("processing column: {}".format(col1))
        tmp_df = expanded_df[[col1]]
        colnames = [col1 + "_" + str(kk) for kk in range(2,13)] # matches the colnames of the all_df
        for idid, values in tmp_df.iterrows():
            if np.any(pd.isnull(values)):
                continue
            for idx3, value in enumerate(values[0]):
                all_df.loc[idid,colnames[idx3]] = value
    expanded_df = pd.concat([expanded_df,all_df],axis=1)
    print("expanded_df size after extracting distance", expanded_df.shape)

    for col2 in expanded_df.columns:
        if expanded_df[col2].dtype not in [int, float]:
            if col2 == "inroi_type2":
                continue
            if "c2t_distance_type" in col2:
                continue
            expanded_df.drop(columns=col2,inplace=True)
    print("expanded_df size after dropping some columns", expanded_df.shape)
    # drop cols with inroi in them
    inroi_cols = [col3 for col3 in expanded_df.columns if 'inroi_type' in col3]
    inroi_cols = inroi_cols[1:] # keep inroi_type2
    expanded_df = expanded_df.drop(columns=inroi_cols)
    for col3 in expanded_df.columns:
        if col3 in "c2t_distance_type" and len(col3.split("_")) == 3: #drop the original c2t_distance_types, but keep the c2t_distance_type2_2.
            expanded_df.drop(columns=col3,inplace=True)
    print("expanded_df size after dropping some columns2", expanded_df.shape)
    print("calculating average")
    # groupby and average now:
    expanded_df = expanded_df.replace([np.inf, -np.inf, np.nan], np.nan) # replace inf -inf and nan wit han's
    print("expanded_df size after dropping nan's", expanded_df.shape)
    grouped = expanded_df.groupby('inroi_type2')
    grouped_list = [grouped.get_group(x) for x in grouped.groups]

    finalavgdf = pd.DataFrame()
    finalstddf = pd.DataFrame()
    for idx4 in range(len(grouped_list)): #
        groupavgdf = pd.DataFrame(grouped_list[idx4].mean(skipna=True)).T #nanmean
        finalavgdf = pd.concat([finalavgdf,groupavgdf])
        groupstddf = pd.DataFrame(grouped_list[idx4].std(skipna=True)).T #nanstd
        finalstddf = pd.concat([finalstddf,groupstddf])
    print("shape of avg df is",finalavgdf.shape)
    print("shape of std df is",finalstddf.shape)

    # save average df
    print("saving excel")
    with pd.ExcelWriter(output_pth, engine='openpyxl', mode='a') as writer:
        finalavgdf.to_excel(writer, sheet_name='averages', index=False)
        finalstddf.to_excel(writer, sheet_name='std', index=False)

Pickle File Processed:   0%|          | 0/214 [00:00<?, ?it/s]

converting pkl to df:
extracting distance
empty row number 4015
processing column: c2t_distance_type2
processing column: c2t_distance_type3
processing column: c2t_distance_type4
processing column: c2t_distance_type5
processing column: c2t_distance_type6
processing column: c2t_distance_type7
processing column: c2t_distance_type8
processing column: c2t_distance_type9
processing column: c2t_distance_type10
processing column: c2t_distance_type11
expanded_df size after extracting distance (4015, 350)
expanded_df size after dropping some columns (4015, 280)
expanded_df size after dropping some columns2 (4015, 271)
calculating average
expanded_df size after dropping nan's (4015, 271)


C:\Users\Kevin\AppData\Local\Temp\ipykernel_21568\435151115.py:77: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  groupavgdf = pd.DataFrame(grouped_list[idx4].mean(skipna=True)).T #nanmean
C:\Users\Kevin\AppData\Local\Temp\ipykernel_21568\435151115.py:79: FutureWarning: The default value of numeric_only in DataFrame.std is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  groupstddf = pd.DataFrame(grouped_list[idx4].std(skipna=True)).T


shape of avg df is (1, 261)
shape of std df is (1, 261)
saving excel


Pickle File Processed:   0%|          | 1/214 [00:26<1:34:35, 26.65s/it]

...

shape of avg df is (4, 270)
shape of std df is (4, 270)
saving excel


Pickle File Processed:   1%|          | 2/214 [07:01<14:19:52, 243.36s/it]

converting pkl to df:
extracting distance
empty row number 8344
processing column: c2t_distance_type2
processing column: c2t_distance_type3
processing column: c2t_distance_type4
processing column: c2t_distance_type5
processing column: c2t_distance_type6
processing column: c2t_distance_type7
processing column: c2t_distance_type8
processing column: c2t_distance_type9
processing column: c2t_distance_type10
processing column: c2t_distance_type11
expanded_df size after extracting distance (8344, 350)
expanded_df size after dropping some columns (8344, 279)
expanded_df size after dropping some columns2 (8344, 271)
calculating average
expanded_df size after dropping nan's (8344, 271)


C:\Users\Kevin\AppData\Local\Temp\ipykernel_21568\435151115.py:77: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  groupavgdf = pd.DataFrame(grouped_list[idx4].mean(skipna=True)).T #nanmean
C:\Users\Kevin\AppData\Local\Temp\ipykernel_21568\435151115.py:79: FutureWarning: The default value of numeric_only in DataFrame.std is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  groupstddf = pd.DataFrame(grouped_list[idx4].std(skipna=True)).T
C:\Users\Kevin\AppData\Local\Temp\ipykernel_21568\435151115.py:79: FutureWarning: The default value of numeric_only in DataFrame.std is deprecated. In a future version, it

shape of avg df is (4, 267)
shape of std df is (4, 267)
saving excel


### Now merge all 214 excel sheets of average together to create a one, large dataframe that we can use to concatenate to the overall clue data df. Note that not all of the excel sheets have the same # of columns, since columns with all NAN's get dropped. Just merge by row!

In [ ]:
ind_xl_src = r'\\shelter\Kyu\skin_aging\clue_cohort\wsi\hovernet_out\df\output_excel'
ind_xl_name = [x for x in os.listdir(ind_xl_src) if x.endswith(".xlsx")]
ind_xl_path = [os.path.join(ind_xl_src,y) for y in ind_xl_name]
total_df = pd.DataFrame()
for _, xlpth in tqdm(enumerate(range(len(ind_xl_path))),desc="Excel File Processed",colour='red',total = len(ind_xl_path)):
    ind_df = pd.read_excel(xlpth,sheet_name='averages')
    total_df = pd.concat([total_df,ind_df],axis=0)
total_df = total_df.reset_index(drop=True)
save_path = r'\\shelter\Kyu\skin_aging\clue_cohort\wsi\hovernet_out\df\output_excel\total_df'
total_df.to_excel(save_path + "total_avg" + ".xlsx")

### Then merge all 214 excel sheets of std together to create one, large dataframe we can use to concatenate to th eoverall clue data df.

In [ ]:
ind_xl_src = r'\\shelter\Kyu\skin_aging\clue_cohort\wsi\hovernet_out\df\output_excel'
ind_xl_name = [x for x in os.listdir(ind_xl_src) if x.endswith(".xlsx")]
ind_xl_path = [os.path.join(ind_xl_src,y) for y in ind_xl_name]
total_df = pd.DataFrame()
for _, xlpth in tqdm(enumerate(range(len(ind_xl_path))),desc="Excel File Processed",colour='red',total = len(ind_xl_path)):
    ind_df = pd.read_excel(xlpth,sheet_name='std')
    total_df = pd.concat([total_df,ind_df],axis=0)
total_df = total_df.reset_index(drop=True)
save_path = r'\\shelter\Kyu\skin_aging\clue_cohort\wsi\hovernet_out\df\output_excel\total_df'
total_df.to_excel(save_path + "total_std" + ".xlsx")

### Then manually copy paste these to the overall clue data df!